In [1]:
from keras.datasets import cifar10
from keras.layers import Input, Dense,Conv2D,MaxPooling2D,UpSampling2D,BatchNormalization
from keras.models import Model,Sequential
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
(X_train, _), (X_test, _) = cifar10.load_data()
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255
# X_train = X_train.reshape(len(X_train),X_train.shape[1],X_train.shape[2],3)
# X_test = X_test.reshape(len(X_test), X_test.shape[1],X_test.shape[2],3)
print(X_train.shape)
print(X_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [5]:
input_img = Input(shape=(32,32,3))

In [1]:
model = Sequential([
                    # encoder network
                    Conv2D(32, 3, activation='relu', padding='same', input_shape=(28, 28, 1)),
                    MaxPooling2D(2, padding='same'),
                    Conv2D(16, 3, activation='relu', padding='same'),
                    MaxPooling2D(2, padding='same'),
                    # decoder network
                    Conv2D(16, 3, activation='relu', padding='same'),
                    UpSampling2D(2),
                    Conv2D(32, 3, activation='relu', padding='same'),
                    UpSampling2D(2),
                    # output layer
                    Conv2D(1, 3, activation='sigmoid', padding='same')
])

model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

NameError: name 'Sequential' is not defined

In [11]:
def build_autoencoder(input_img):
    #encoder
    encoder = Conv2D(16, kernel_size=3, activation='relu', padding='same')(input_img)
    encoder = MaxPooling2D(2, padding='same')(encoder)
    encoder = Conv2D(8, kernel_size=3, activation='relu', padding='same')(encoder)
    encoder = MaxPooling2D(2, padding='same')(encoder)
    encoder = Conv2D(8, kernel_size=3, activation='relu', padding='same')(encoder)
    encoded = MaxPooling2D(2, padding='same', name='encoder')(encoder)

    #decoder
    decoder = Conv2D(8, kernel_size=3, activation='relu', padding='same')(encoded)
    decoder = UpSampling2D(2)(decoder)
    decoder = Conv2D(8, kernel_size=3, activation='relu', padding='same')(decoder)
    decoder = UpSampling2D(2)(decoder)
    decoder = Conv2D(16, kernel_size=3, activation='relu',padding='same')(decoder)
    decoder = UpSampling2D(2)(decoder)
    decoded = Conv2D(3, kernel_size=3, activation='sigmoid', padding='same')(decoder)

    return encoded, decoded

In [14]:
IMG_SHAPE = X_test.shape[1:]
encoder, decoder = build_autoencoder(input_img)


# inp = Input(IMG_SHAPE)
# code = encoder(inp)
# reconstruction = decoder(code)
autoencoder = Model(input_img,decoder)
autoencoder.compile(optimizer='adam', loss='mse')
print(autoencoder.summary())

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_23 (Conv2D)          (None, 32, 32, 16)        448       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 16, 16)       0         
 2D)                                                             
                                                                 
 conv2d_24 (Conv2D)          (None, 16, 16, 8)         1160      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 8, 8, 8)          0         
 2D)                                                             
                                                                 
 conv2d_25 (Conv2D)          (None, 8, 8, 8)           584 

In [3]:
model = Sequential()

model.add(Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=(32, 32, 3)))
model.add(BatchNormalization())     # 32x32x32
model.add(Conv2D(32, kernel_size=3, strides=2, padding='same', activation='relu'))      # 16x16x32
model.add(Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu'))      # 16x16x32
model.add(BatchNormalization())     # 16x16x32
model.add(UpSampling2D())
model.add(Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu'))      # 32x32x32
model.add(BatchNormalization())
model.add(Conv2D(3,  kernel_size=1, strides=1, padding='same', activation='sigmoid'))   # 32x32x3

model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 16, 16, 32)       128       
 hNormalization)                                                 
                                                                 
 up_sampling2d (UpSampling2D  (None, 32, 32, 32)       0

2022-07-24 21:40:29.901860: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
autoencoder=Model(input_img, decoded)

NameError: name 'input_img' is not defined